In [237]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [238]:
hcv =pd.read_csv('/Users/tokarev/Documents/programmes/DataScale/Machine Learning/Projet/Data/hcvdat0.csv')

In [239]:
mapping_cat = {'0=Blood Donor' : 0, '1=Hepatitis' : 1, '2=Fibrosis' : 2, '3=Cirrhosis' : 3}
mapping_sex = {'m' : 0, 'f' :1}
hcv = hcv.replace({'Category': mapping_cat})
hcv = hcv.replace({'Sex': mapping_sex})

In [240]:
#HANDLING NAN VALUES
blooddonors = hcv.loc[hcv['Category'] == 0]
hepatitis = hcv.loc[hcv['Category'] == 1]
fibrosis = hcv.loc[hcv['Category'] == 2]
cirrhosis = hcv.loc[hcv['Category'] == 3]
means_bd = []
means_hep = []
means_fib = []
means_cir = []
for i in range(4,len(hcv.columns)) :
    #print("%s : %d " % (hcv.columns[i], i-4))
    means_bd.append(hcv.loc[hcv['Category'] == 0][hcv.columns[i]].mean())
    means_hep.append(hcv.loc[hcv['Category'] == 1][hcv.columns[i]].mean())
    means_fib.append(hcv.loc[hcv['Category'] == 2][hcv.columns[i]].mean())
    means_cir.append(hcv.loc[hcv['Category'] == 3][hcv.columns[i]].mean())
    blooddonors[blooddonors.columns[i]] = blooddonors[blooddonors.columns[i]].fillna(value = means_bd[i-4])
    hepatitis[hepatitis.columns[i]] = hepatitis[hepatitis.columns[i]].fillna(value = means_hep[i-4])
    fibrosis[fibrosis.columns[i]] = fibrosis[fibrosis.columns[i]].fillna(value = means_fib[i-4])
    cirrhosis[cirrhosis.columns[i]] = cirrhosis[cirrhosis.columns[i]].fillna(value = means_cir[i-4])
frames = [blooddonors, hepatitis, fibrosis, cirrhosis]
hcv = pd.concat(frames)

In [241]:
hcv.Category.value_counts()

0    533
3     30
1     24
2     21
Name: Category, dtype: int64

In [242]:
#iloc[x:x, y:y]
Y = hcv.Category
X = hcv.iloc[:,2:]
X.head(5)

,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
0,32,0,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
1,32,0,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
2,32,0,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
3,32,0,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
4,32,0,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7


In [243]:
# exemple pour Category ou Sex car pas int mais deja fait
#enc = LabelEncoder()
#X.loc[:,['Sex']] = \      \ juste pour next line
#X.loc[:,['Sex']].apply(enc.fit_transform)

In [262]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.3, random_state=30)
model = KNeighborsClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
print(accuracy_score(y_test, y_predict))
pd.crosstab(y_test, y_predict)

0.9180327868852459


col_0,0,1,3
Category,,,
0,161,1,0
1,3,2,1
2,5,3,0
3,2,0,5


In [263]:
########################
#                      #
# pip install imblearn #
#                      #
########################

In [264]:
from imblearn.over_sampling import SMOTE

In [265]:
smote = SMOTE()
x_train_smote, y_train_smote = smote.fit_sample(x_train, y_train)

In [266]:
from collections import Counter
print("Before SMOTE : ", Counter(y_train))
print("After SMOTE : ", Counter(y_train_smote))

Before SMOTE :  Counter({0: 371, 3: 23, 1: 18, 2: 13})
After SMOTE :  Counter({0: 371, 2: 371, 1: 371, 3: 371})


In [267]:
model.fit(x_train_smote, y_train_smote)
y_predict = model.predict(x_test)
print(accuracy_score(y_test, y_predict))
pd.crosstab(y_test, y_predict)

0.8961748633879781


col_0,0,1,2,3
Category,,,,
0,150,6,2,4
1,0,3,2,1
2,1,2,5,0
3,0,0,1,6
